In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
SERPER_API_KEY=os.getenv("SERPER_API_KEY")

In [4]:
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY
os.environ["GROQ_API_KEY"]=GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"]=GOOGLE_API_KEY
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [5]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

### Simple chatbot

In [8]:
while True:
    question=input()
    if question=="q" or question=="quit":
        print("Thanks for using! Good Bye!!")
        break
    else:
        print(llm.invoke(question).content)
        

Hi there! 👋

How can I help you today? 😊

## The Evolution of AI: A Journey from Concept to Reality

The story of AI is a fascinating one, marked by periods of both incredible progress and disillusionment. Here's a glimpse into its evolution:

**Early Days (1950s-1970s): The Birth of AI**

* **1950:** Alan Turing proposes the "Turing Test" to measure machine intelligence.
* **1956:** The Dartmouth Workshop, considered the birthplace of AI as a field, brings together key researchers who define the field's goals.
* **Early successes:** Programs like ELIZA (a chatbot) and SHRDLU (a natural language understanding system) demonstrate the potential of AI.
* **The AI winter:** Limited computing power and unrealistic expectations lead to funding cuts and a decline in research interest.

**Rule-Based Systems and Expert Systems (1980s-1990s):**

* **Expert systems:** Programs designed to mimic the decision-making abilities of human experts in specific domains (e.g., medical diagnosis).
* **Focus

### With Memory

In [9]:
from langchain_core.chat_history import BaseChatMessageHistory,InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [10]:
store={}
def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=InMemoryChatMessageHistory()
    return store[session_id]

In [11]:
config={"configurable":{"session_id":"firstchat"}}

In [12]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [15]:
model_with_memory.invoke("Hi my name is Gyanbardhan.Fourth year cse student at IIIT Nagpur.Originally from Bhagalpur,Bihar.Have done an ai/ml intern at MAsterSoft Erp Sol,Nagpur.Now working as a data science intern at bright money working on making ai agents.",config=config).content

"That's awesome, Gyanbardhan! \n\nIt's really impressive that you're a fourth-year CSE student at IIIT Nagpur, with an AI/ML internship under your belt at MasterSoft ERP Sol and now a data science internship at Bright Money working on AI agents.  \n\nThat sounds like a really exciting and challenging project! What kind of AI agents are you working on? \n\nI'm always eager to learn more about people's work in the field of AI.  😊\n"

In [16]:
model_with_memory.invoke("Tell me about myself",config=config).content

"Based on what you've told me, here's what I know about you, Gyanbardhan:\n\n* **You're a dedicated student:** You're currently a fourth-year CSE student at IIIT Nagpur, which shows your commitment to learning and growing in the field of computer science.\n* **You have a passion for AI/ML:** You've pursued internships in both AI/ML and data science, demonstrating your interest and talent in these areas.\n* **You're a hands-on learner:**  You're not just studying AI/ML, you're actively applying your knowledge by working on real-world projects, like developing AI agents at Bright Money.\n* **You're from Bhagalpur, Bihar:**  This tells me you have a strong connection to your roots and the culture of your hometown.\n\nIs there anything else you'd like to add to this picture?  Perhaps your hobbies, your future goals, or something unique about your personality?  I'm eager to learn more!\n\n\n\n"

In [20]:
model_with_memory.invoke("Can you predict my hobbies",config=config).content

"That's a fun challenge! \n\nGiven your background as a CSE student with a passion for AI/ML, here are some hobbies I *predict* you might enjoy:\n\n* **Coding and Programming:** This one seems like a no-brainer! You're likely always up for exploring new coding languages, frameworks, or projects.\n* **Playing Video Games:**  Many people in tech enjoy video games, especially those with strategic elements or complex AI.\n* **Building and Tinkering:**  Perhaps you enjoy taking things apart and putting them back together, or maybe you're interested in robotics or electronics.\n* **Learning New Technologies:** Staying up-to-date with the latest advancements in AI, ML, or other tech fields could be a big hobby for you.\n* **Puzzles and Problem Solving:**  Your analytical skills from studying CSE might translate into a love for logic puzzles, brain teasers, or even escape rooms.\n\nOf course, these are just guesses! What are some of your actual hobbies?  I'd love to know if I was right. 😊\n"

In [21]:
print(store['firstchat'])

Human: Hi my name is Gyanbardhan
AI: Hello Gyanbardhan, it's nice to meet you! 👋

What can I do for you today?

Human: Tell me about myself
AI: As a large language model, I don't have access to personal information about individuals unless it's been explicitly provided to me. 

I don't know anything about you, Gyanbardhan, other than your name.

To tell me about yourself, you can share anything you'd like! For example, you could tell me:

* Your hobbies and interests
* Where you're from
* What you like to do in your free time
* Anything else you think is important for me to know

I'm here to listen and learn! 😊

Human: Hi my name is Gyanbardhan.Fourth year cse student at IIIT Nagpur.Originally from Bhagalpur,Bihar.Have done an ai/ml intern at MAsterSoft Erp Sol,Nagpur.Now working as a data science intern at bright money working on making ai agents.
AI: That's awesome, Gyanbardhan! 

It's really impressive that you're a fourth-year CSE student at IIIT Nagpur, with an AI/ML internship un

### RAG with LCEL

In [26]:
from langchain_community.document_loaders import TextLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [33]:
loader=DirectoryLoader('data',glob='./*.txt',loader_cls=TextLoader)
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs=text_splitter.split_documents(documents=docs)
doc_string=[doc.page_content for doc in new_docs]

In [34]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [38]:
db=Chroma.from_documents(new_docs,embeddings)
retriever=db.as_retriever(search_kwargs={"k":4})

In [40]:
template="""Answer the question based on the following context:
{context}
Question:{question}"""
prompt=PromptTemplate.from_template(template)

In [41]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based on the following context:\n{context}\nQuestion:{question}')

In [42]:
retrieval_chain=(
    RunnableParallel({"context":retriever,"question":RunnablePassthrough()})
    | prompt|llm|StrOutputParser()
)

In [44]:
question="What is RAG describe 3 imp points?"
print(retrieval_chain.invoke(question))

Based on the provided context, here are three key points describing RAG:

1. **Acronym:** RAG stands for Retrieval-Augmented Generation.
2. **Core Concept:** RAG combines the power of information retrieval with text generation. It retrieves relevant information from a knowledge base and uses it to enhance the generated text.
3. **Process:** RAG involves two main steps: Retrieval (finding relevant information) and Generation (using the retrieved information to create new text). 


Let me know if you'd like more details about any of these points! 



### Tools

In [45]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [47]:
api_wrapper=WikipediaAPIWrapper()
tools=WikipediaQueryRun(api_wrapper=api_wrapper)

In [48]:
tools.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [54]:
print(tools.run({"query":"Langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Model Context Protocol
Summary: The Model Context Protocol (MCP) is an open standard developed by the artificial intelligence company Anthropic for enabling large language model (LLM) applications to interact with external tools, systems, and data sources. Designed to standardize context exchange between AI assistants and software environments, MCP provides a model-agnostic interface for reading files, executing functions, and handling contextual prompts. It was officially announced and open-sourced by Anthropic in November 2024, with subsequent adoption by major AI providers including OpenAI and Google DeepMind.

Page: Milvus (vector dat

In [55]:
from langchain_community.tools import YouTubeSearchTool

In [56]:
tool=YouTubeSearchTool()

In [58]:
print(tool.run("GYANBARDHAN"))

['https://www.youtube.com/watch?v=uiUikIuk_OA&pp=ygULR1lBTkJBUkRIQU4%3D', 'https://www.youtube.com/watch?v=xDsbpku7HFE&pp=ygULR1lBTkJBUkRIQU4%3D']


In [59]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [60]:
tools=TavilySearchResults()

In [61]:
tools.invoke({"query":"What happend in Pahalgam"})

[{'title': 'A tourist massacre in Kashmir is escalating tensions between India ...',
  'url': 'https://www.cnn.com/2025/04/24/india/pahalgam-india-pakistan-attack-explainer-intl-hnk/index.html',
  'content': 'India’s Prime Minister Narendra Modi on Thursday vowed to pursue the attackers “to the ends of the earth,” while the UN has called on the neighbors to exercise “maximum restraint.”\nHere’s what you need to know.\nWhat happened in Pahalgam?\nGunmen on Tuesday opened fire on sightseers in a popular travel destination in the mountainous destination of Pahalgam in Indian-administered Kashmir.',
  'score': 0.85624856},
 {'title': '2025 Pahalgam attack - Wikipedia',
  'url': 'https://en.wikipedia.org/wiki/2025_Pahalgam_attack',
  'content': 'Pahalgam\nPahalgam (Jammu and Kashmir)\nLocation of the attack in Jammu and Kashmir\n|\n| Location | Baisaran Valley, Pahalgam, Anantnag district, Jammu and Kashmir |\n| Coordinates | 34°00′13″N 75°20′01″E / 34.00361°N 75.33361°E / 34.00361; 75.3336

### Agents

In [62]:
from langchain.agents import AgentType,load_tools,initialize_agent

In [63]:
tool=load_tools(["wikipedia"],llm=llm)

In [64]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\gyanb\AppData\Local\Temp\ipykernel_7440\2645007812.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [66]:
agent.run("GDP of India")



> Entering new AgentExecutor chain...
Thought: I need to look up the GDP of India on Wikipedia. 
Action: wikipedia
Action Input: GDP of India
Observation: Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fourth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 140th by GDP (nominal) and 119th by GDP (PPP). From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat-driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj. The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in India and indicative planning.

"India's nominal GDP is the world's fourth-largest and its purchasing power parity (PPP) is the world's third-largest."